# Gaussian1dModel Examples

### Examples taken from QuantLib's C++ Example (by Peter Caspers) translated to Python

This is some example code showing how to use the GSR (Gaussian short rate) and Markov Functional model.

In [1]:
import QuantLib as ql

In [2]:
refDate = ql.Date(30,4,2014)

In [3]:
ql.Settings.instance().evaluationDate=refDate

In [4]:
print("EvalDate: ",ql.Settings.instance().evaluationDate)

EvalDate:  April 30th, 2014


In [5]:
forward6mLevel = 0.025
oisLevel = 0.02

In [6]:
forward6mQuote = ql.QuoteHandle(ql.SimpleQuote(forward6mLevel))
oisQuote = ql.QuoteHandle(ql.SimpleQuote(oisLevel))

In [7]:
yts6m = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), forward6mQuote, ql.Actual365Fixed() ))
ytsOis = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), oisQuote, ql.Actual365Fixed() ))

In [8]:
euribor6m = ql.Euribor6M(yts6m)

We assume a multicurve setup, for simplicity with flat yield term structures. The discounting curve is an Eonia curve at a level of {{oisLevel}} and the forwarding curve is an Euribior 6m curve at a level of {{forward6mLevel}}

In [9]:
volLevel = 0.20

In [10]:
volQuote = ql.QuoteHandle(ql.SimpleQuote(volLevel))

In [11]:
swaptionVol = ql.ConstantSwaptionVolatility(0, ql.TARGET(), ql.ModifiedFollowing, volQuote,
                                                                                ql.Actual365Fixed())

In [12]:
swaptionVolHandle = ql.SwaptionVolatilityStructureHandle(ql.ConstantSwaptionVolatility(0, ql.TARGET(), ql.ModifiedFollowing, volQuote,
                                                                                ql.Actual365Fixed()))

In [13]:
opt_tenors = [ql.Period(1,ql.Years),ql.Period(2,ql.Years)]
swap_tenors = [ql.Period(1,ql.Years),ql.Period(2,ql.Years)]
atm_quotes = [[ql.QuoteHandle(ql.SimpleQuote(0.01))]*2 for i in opt_tenors]


In [14]:
atm_quotes

[[<QuantLib.QuantLib.QuoteHandle; proxy of <Swig Object of type 'std::vector< Handle< Quote > >::value_type *' at 0x00000253A8EB6900> >,
  <QuantLib.QuantLib.QuoteHandle; proxy of <Swig Object of type 'std::vector< Handle< Quote > >::value_type *' at 0x00000253A8EB6900> >],
 [<QuantLib.QuantLib.QuoteHandle; proxy of <Swig Object of type 'std::vector< Handle< Quote > >::value_type *' at 0x00000253A8EB6930> >,
  <QuantLib.QuantLib.QuoteHandle; proxy of <Swig Object of type 'std::vector< Handle< Quote > >::value_type *' at 0x00000253A8EB6930> >]]

In [15]:
swaptionVol = ql.SwaptionVolatilityMatrix(ql.TARGET(), ql.ModifiedFollowing, opt_tenors, swap_tenors,
                                    atm_quotes, ql.Actual365Fixed(), True, ql.ShiftedLognormal)

In [53]:
swaptionVolHandle = ql.SwaptionVolatilityStructureHandle(swaptionVol)

In [90]:
swaptionVolCube = ql.SwaptionVolCube2(swaptionVolHandle, opt_tenors, swap_tenors, [-0.01,  0.01],
                                     [[ql.QuoteHandle(ql.SimpleQuote(0.01)), ql.QuoteHandle(ql.SimpleQuote(0.01))] for o in opt_tenors for s in swap_tenors],
                                      ql.EuriborSwapIsdaFixA(ql.Period("10Y"), ytsOis), ql.EuriborSwapIsdaFixA(ql.Period("1Y"), ytsOis),
                                     True)

In [16]:
swaptionVolHandle = ql.SwaptionVolatilityStructureHandle(swaptionVol)

For the volatility we assume a flat swaption volatility at {{volLevel}}

In [17]:
strike = 0.04

We consider a standard 10y bermudan payer swaption with yearly exercises at a strike of {{strike}}

In [18]:
effectiveDate = ql.TARGET().advance(refDate, ql.Period(2,ql.Days))

In [19]:
maturityDate = ql.TARGET().advance(effectiveDate, ql.Period(10,ql.Years))

In [20]:
maturityDate

Date(6,5,2024)

In [21]:
fixedSchedule = ql.Schedule(effectiveDate, maturityDate, ql.Period(1,ql.Years), ql.TARGET(),
                            ql.ModifiedFollowing, ql.ModifiedFollowing,
                            ql.DateGeneration.Forward, False)

In [22]:
for i in fixedSchedule:
    print(i)

May 5th, 2014
May 5th, 2015
May 5th, 2016
May 5th, 2017
May 7th, 2018
May 6th, 2019
May 5th, 2020
May 5th, 2021
May 5th, 2022
May 5th, 2023
May 6th, 2024


In [23]:
floatingSchedule = ql.Schedule(effectiveDate, maturityDate, ql.Period(6,ql.Months), ql.TARGET(),
                            ql.ModifiedFollowing, ql.ModifiedFollowing,
                            ql.DateGeneration.Forward, False)

In [24]:
for i in floatingSchedule:
    print(i)

May 5th, 2014
November 5th, 2014
May 5th, 2015
November 5th, 2015
May 5th, 2016
November 7th, 2016
May 5th, 2017
November 6th, 2017
May 7th, 2018
November 5th, 2018
May 6th, 2019
November 5th, 2019
May 5th, 2020
November 5th, 2020
May 5th, 2021
November 5th, 2021
May 5th, 2022
November 7th, 2022
May 5th, 2023
November 6th, 2023
May 6th, 2024


In [25]:
underlying = ql.NonstandardSwap(ql.VanillaSwap.Payer, [1.0]*(len(fixedSchedule)-1), [1.0]*(len(floatingSchedule)-1), fixedSchedule,
                                              [strike]*(len(fixedSchedule)-1), ql.Thirty360(), floatingSchedule, 
                                              euribor6m, [1.0]*(len(floatingSchedule)-1), [0.0]*(len(floatingSchedule)-1), ql.Actual360())

In [26]:
for cf in underlying.fixedLeg():
    print (cf.date(), cf.amount())

May 5th, 2015 0.040000000000000036
May 5th, 2016 0.040000000000000036
May 5th, 2017 0.040000000000000036
May 7th, 2018 0.04022222222222216
May 6th, 2019 0.03988888888888886
May 5th, 2020 0.03988888888888886
May 5th, 2021 0.040000000000000036
May 5th, 2022 0.040000000000000036
May 5th, 2023 0.040000000000000036
May 6th, 2024 0.04011111111111121


In [27]:
for cf in underlying.floatingLeg():
    print (cf.date(), cf.amount())

November 5th, 2014 0.012682488917620915
May 5th, 2015 0.012474424851870225
November 5th, 2015 0.012682488917621138
May 5th, 2016 0.012543774790186868
November 7th, 2016 0.012821222048292169
May 5th, 2017 0.012335739224410558
November 6th, 2017 0.012751853107389575
May 7th, 2018 0.012543774790186868
November 5th, 2018 0.012543774790186868
May 6th, 2019 0.012543774790186868
November 5th, 2019 0.012613129478662222
May 5th, 2020 0.012543774790186868
November 5th, 2020 0.012682488917621138
May 5th, 2021 0.012474424851870225
November 5th, 2021 0.012682488917621138
May 5th, 2022 0.012474424851870003
November 7th, 2022 0.012821222048292391
May 5th, 2023 0.012335739224410558
November 6th, 2023 0.012751853107389575
May 6th, 2024 0.012543774790186868


In [28]:
exerciseDates = [ql.TARGET().advance(fixedSchedule[i], ql.Period(-2, ql.Days)) for i in range(1,10)]

In [29]:
exerciseDates

[Date(30,4,2015),
 Date(3,5,2016),
 Date(3,5,2017),
 Date(3,5,2018),
 Date(2,5,2019),
 Date(30,4,2020),
 Date(3,5,2021),
 Date(3,5,2022),
 Date(3,5,2023)]

In [30]:
exercise = ql.BermudanExercise(exerciseDates, False)

In [31]:
swaption = ql.NonstandardSwaption(underlying, exercise)

The model is a one factor Hull White model with piecewise volatility adapted to our exercise dates.

In [32]:
stepDates = [exerciseDates[i] for i in range(0, len(exerciseDates)-1)]

In [33]:
sigmas = [ql.QuoteHandle(ql.SimpleQuote(0.01)) for i in range(0, len(stepDates)+1)]

In [34]:
reversion = 0.01

In [35]:
reversions = [ql.QuoteHandle(ql.SimpleQuote(reversion))]

The reversion is just kept constant at a level of {{reversion}}

The model's curve is set to the 6m forward curve. Note that the model adapts automatically to other curves where appropriate (e.g. if an index requires a different forwarding curve) or where explicitly specified (e.g. in a swaption pricing engine).

In [36]:
gsr = ql.Gsr(yts6m, stepDates, sigmas, reversions)

In [37]:
swaptionEngine = ql.Gaussian1dSwaptionEngine(gsr, 64, 7.0, True, False, ytsOis)

In [38]:
nonstandardSwaptionEngine = ql.Gaussian1dNonstandardSwaptionEngine(gsr, 64, 7.0, True, False, ql.QuoteHandle(),ytsOis, ql.Gaussian1dNonstandardSwaptionEngine.Naive)

In [39]:
swaption.setPricingEngine(nonstandardSwaptionEngine)

The engine can generate a calibration basket in two modes. The first one is called Naive and generates ATM swaptions adapted to the exercise dates of the swaption and its maturity date.

In [40]:
swapBase = ql.EuriborSwapIsdaFixA(ql.Period(10,ql.Years), yts6m, ytsOis)

In [41]:
basket = swaption.calibrationBasket(swapBase, swaptionVol, "Naive")

In [102]:
h=ql.SwaptionVolatilityStructureHandle(swaptionVolCube)

In [107]:
basket = swaption.calibrationBasket(swapBase, h.thisown, "Naive")

TypeError: in method 'NonstandardSwaption_calibrationBasket', argument 3 of type 'boost::shared_ptr< SwaptionVolatilityStructure >'

In [82]:
def printBasket(basket):
    print()
    print('{:<20}{:<20}{:<20}{:<14}{:<12}{:<14}'.format("Expiry", "Maturity", "Nominal", "Rate", "Pay/Rec", "Market ivol"))
    print("===================="*4+"==================")
    for helper in basket:
        helper = ql.as_swaption_helper(helper)
        endDate = helper.swaptionMaturityDate()
        nominal = helper.swaptionNominal()
        vol = helper.volatility().value()
        rate = helper.swaptionStrike()
        expiry = helper.swaptionExpiryDate()
        swp_type = 0 # to be SWIGed
        print('{:<20}{:<20}{:<20}{:<14}{:<12}{:<14}'.format(str(expiry), str(endDate), ("%.2f"%nominal), ("%.6f"%rate), str(swp_type), ("%.6f"%vol)))

The resulting basket looks as follows:

In [92]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    May 6th, 2024       1.00                0.025307      0           0.020000      
May 3rd, 2016       May 6th, 2024       1.00                0.025300      0           0.020000      
May 3rd, 2017       May 6th, 2024       1.00                0.025303      0           0.020000      
May 3rd, 2018       May 6th, 2024       1.00                0.025306      0           0.020000      
May 2nd, 2019       May 6th, 2024       1.00                0.025311      0           0.020000      
April 30th, 2020    May 6th, 2024       1.00                0.025300      0           0.020000      
May 3rd, 2021       May 6th, 2024       1.00                0.025306      0           0.020000      
May 3rd, 2022       May 6th, 2024       1.00                0.025318      0           0.020000      
May 3rd, 2023       May 6th, 2024       1.00                0.025353      0           0.02

Let's calibrate our model to this basket. We use a specialized calibration method calibrating the sigma function one by one to the calibrating vanilla swaptions. The result of this is as follows:

In [95]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [96]:
method = ql.LevenbergMarquardt()

In [97]:
ec = ql.EndCriteria(1000, 10, 1e-8, 1e-8, 1e-8)

In [98]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [99]:
def printModelCalibration(basket, volatility):
    print()
    print('{:<20}{:<14}{:<20}{:<20}{:<14}{:<14}'.format("Expiry", "Model sigma", "Model price", "Market price", "Model ivol", "Market ivol"))
    print("===================="*5)
    for i in range(0,len(basket)):
        helper = basket[i]
        helper = ql.as_swaption_helper(helper)
        expiry = helper.swaptionExpiryDate()
        print('{:<20}{:<14}{:<20}{:<20}{:<14}{:<14}'.format(str(expiry), ("%.6f"%volatility[i]), ("%.6f"%helper.modelValue()),
                                                           ("%.6f"%helper.marketValue()), ("%.6f"%helper.impliedVolatility(helper.modelValue(), 1e-6, 1000, 0.0, 2.0)),
                                                           ("%.6f"%helper.volatility().value())))
    if (len(volatility) > len(basket)):
        print('{:<20}'.format(("%.6f"%volatility[-1])))
        

In [100]:
printModelCalibration(basket, gsr.volatility())


Expiry              Model sigma   Model price         Market price        Model ivol    Market ivol   
April 30th, 2015    0.000519      0.001614            0.001614            0.020000      0.020000      
May 3rd, 2016       0.000519      0.002013            0.002013            0.020000      0.020000      
May 3rd, 2017       0.000518      0.002133            0.002133            0.020000      0.020000      
May 3rd, 2018       0.000519      0.002087            0.002087            0.020000      0.020000      
May 2nd, 2019       0.000519      0.001925            0.001925            0.020000      0.020000      
April 30th, 2020    0.000519      0.001670            0.001670            0.020000      0.020000      
May 3rd, 2021       0.000519      0.001341            0.001341            0.020000      0.020000      
May 3rd, 2022       0.000519      0.000947            0.000947            0.020000      0.020000      
May 3rd, 2023       0.000516      0.000498            0.000498          

Finally we price our bermudan swaption in the calibrated model:

In [46]:
print("Bermudan swaption NPV (ATM calibrated GSR) = %.6f" % swaption.NPV())

Bermudan swaption NPV (ATM calibrated GSR) = 0.003808


In [47]:
probResult0 = swaption.probabilities()
print("Call probabilities (Naive):")
for prob in probResult0:
    print ("%.4f" % prob)

Call probabilities (Naive):
0.0000
0.0009
0.0045
0.0063
0.0096
0.0142
0.0138
0.0151
0.0166
0.9192


There is another mode to generate a calibration basket called MaturityStrikeByDeltaGamma. This means that the maturity, the strike and the nominal of the calibrating swaption are computed such that the npv and its first and second derivative with respect to the model's state variable) of the exotics underlying match with the calibrating swaption'sunderlying. Let's try this in our case.

In [48]:
basket = swaption.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

In [49]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2016       May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2017       May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2018       May 7th, 2024       1.00                0.040000      0           0.200000      
May 2nd, 2019       May 6th, 2024       1.00                0.040000      0           0.200000      
April 30th, 2020    May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2021       May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2022       May 6th, 2024       1.00                0.040000      0           0.200000      
May 3rd, 2023       May 6th, 2024       1.00                0.040000      0           0.20

The calibrated nominal is close to the exotics nominal. The expiries and maturity dates of the vanillas are the same as in the case above. The difference is the strike which is now equal to the exotics strike.

Let's see how this affects the exotics npv. The recalibrated model is:

In [50]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [51]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [52]:
printModelCalibration(basket, gsr.volatility())


Expiry              Model sigma   Model price         Market price        Model ivol    Market ivol   
April 30th, 2015    0.006508      0.000191            0.000191            0.200000      0.200000      
May 3rd, 2016       0.006502      0.001412            0.001412            0.200000      0.200000      
May 3rd, 2017       0.006480      0.002905            0.002905            0.200000      0.200000      
May 3rd, 2018       0.006464      0.004091            0.004091            0.200000      0.200000      
May 2nd, 2019       0.006422      0.004766            0.004766            0.200000      0.200000      
April 30th, 2020    0.006445      0.004869            0.004869            0.200000      0.200000      
May 3rd, 2021       0.006433      0.004433            0.004433            0.200000      0.200000      
May 3rd, 2022       0.006332      0.003454            0.003454            0.200000      0.200000      
May 3rd, 2023       0.006295      0.001973            0.001973          

And the bermudan's price becomes:

In [53]:
print("Bermudan swaption NPV (deal strike calibrated GSR) = %.6f" % swaption.NPV())

Bermudan swaption NPV (deal strike calibrated GSR) = 0.007627


We can do more complicated things, let's e.g. modify the nominal schedule to be linear amortizing and see what the effect on the generated calibration basket is:

In [54]:
nominalFixed = [ 1.0 - float(i) / (len(fixedSchedule) - 1) for i in range(0,len(fixedSchedule)-1)]
nominalFloating = [ n for n in nominalFixed for repetitions in range(2)]
strikes = [strike]*len(nominalFixed)

In [55]:
underlying2 = ql.NonstandardSwap(ql.VanillaSwap.Payer, nominalFixed, nominalFloating, fixedSchedule,
                                              strikes, ql.Thirty360(), floatingSchedule, 
                                              euribor6m, [1.0]*(len(floatingSchedule)-1), [0.0]*(len(floatingSchedule)-1), ql.Actual360())

In [56]:
swaption2 = ql.NonstandardSwaption(underlying2, exercise)

In [57]:
swaption2.setPricingEngine(nonstandardSwaptionEngine)

In [58]:
basket = swaption2.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

In [59]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    August 5th, 2021    0.72                0.039997      0           0.200000      
May 3rd, 2016       December 6th, 2021  0.64                0.040003      0           0.200000      
May 3rd, 2017       May 5th, 2022       0.56                0.040005      0           0.200000      
May 3rd, 2018       September 7th, 2022 0.49                0.040004      0           0.200000      
May 2nd, 2019       January 6th, 2023   0.41                0.040008      0           0.200000      
April 30th, 2020    May 5th, 2023       0.33                0.039994      0           0.200000      
May 3rd, 2021       September 5th, 2023 0.26                0.039995      0           0.200000      
May 3rd, 2022       January 5th, 2024   0.18                0.040031      0           0.200000      
May 3rd, 2023       May 6th, 2024       0.10                0.040000      0           0.20

The notional is weighted over the underlying exercised into and the maturity is adjusted downwards. The rate on the other hand is not affected.

You can also price exotic bond's features. If you have e.g. a bermudan callable fixed bond you can set up the call right as a swaption to enter into a one leg swap with notional reimbursement at maturity. The exercise should then be written as a rebated exercise paying the notional in case of exercise.

In [60]:
nominalFixed2 = [1.0]*len(nominalFixed)
nominalFloating2 = [0.0]*len(nominalFloating)

In [61]:
underlying3 = ql.NonstandardSwap(ql.VanillaSwap.Receiver, nominalFixed2, nominalFloating2, fixedSchedule,
                                              strikes, ql.Thirty360(), floatingSchedule, 
                                              euribor6m, [1.0]*(len(floatingSchedule)-1), [0.0]*(len(floatingSchedule)-1), ql.Actual360(),
                                False, True)

In [62]:
exercise2 = ql.RebatedExercise(exercise, [-1.0]*len(exerciseDates), 2, ql.TARGET())

In [63]:
swaption3 = ql.NonstandardSwaption(underlying3, exercise2)

In [64]:
oas0 = ql.SimpleQuote(0.0)
oas100 = ql.SimpleQuote(0.01)
oas = ql.RelinkableQuoteHandle(oas0)

In [65]:
nonstandardSwaptionEngine2 = ql.Gaussian1dNonstandardSwaptionEngine(gsr, 64, 7.0, True, False, oas) #change discounting to 6m

In [66]:
swaption3.setPricingEngine(nonstandardSwaptionEngine2)

In [67]:
basket = swaption3.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

The calibration basket looks like this:

In [68]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    April 5th, 2024     0.98                0.039952      0           0.200000      
May 3rd, 2016       April 5th, 2024     0.99                0.039952      0           0.200000      
May 3rd, 2017       May 6th, 2024       0.99                0.039952      0           0.200000      
May 3rd, 2018       May 7th, 2024       0.99                0.039952      0           0.200000      
May 2nd, 2019       May 6th, 2024       0.99                0.039952      0           0.200000      
April 30th, 2020    May 6th, 2024       0.99                0.039951      0           0.200000      
May 3rd, 2021       May 6th, 2024       0.99                0.039951      0           0.200000      
May 3rd, 2022       May 6th, 2024       0.99                0.039952      0           0.200000      
May 3rd, 2023       May 6th, 2024       1.00                0.039949      0           0.20

Note that nominals are not exactly 1.0 here. This is because we do our bond discounting on 6m level while the swaptions are still discounted on OIS level. (You can try this by changing the OIS level to the 6m level, which will produce nominals near 1.0). The npv of the call right is (after recalibrating the model):

In [69]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [70]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [71]:
print("Bond's bermudan call right npv = %.6f" % swaption3.NPV())

Bond's bermudan call right npv = 0.115409


Up to now, no credit spread is included in the pricing. We can do so by specifying an oas in the pricing engine. Let's set the spread level to 100bp and regenerate the calibration basket.

In [72]:
oas.linkTo(oas100) # Credit Spread 100bp

In [73]:
basket = swaption3.calibrationBasket(swapBase, swaptionVol, "MaturityStrikeByDeltaGamma")

In [74]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    February 5th, 2024  0.96                0.029608      0           0.200000      
May 3rd, 2016       March 5th, 2024     0.97                0.029605      0           0.200000      
May 3rd, 2017       April 5th, 2024     0.97                0.029608      0           0.200000      
May 3rd, 2018       April 8th, 2024     0.97                0.029610      0           0.200000      
May 2nd, 2019       April 8th, 2024     0.98                0.029608      0           0.200000      
April 30th, 2020    May 6th, 2024       0.98                0.029612      0           0.200000      
May 3rd, 2021       May 6th, 2024       0.99                0.029609      0           0.200000      
May 3rd, 2022       May 6th, 2024       0.99                0.029603      0           0.200000      
May 3rd, 2023       May 6th, 2024       1.00                0.029586      0           0.20

The adjusted basket takes the credit spread into account. This is consistent to a hedge where you would have a margin on the float leg around 100bp, too.

In [75]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [76]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [77]:
print("Bond's bermudan call right npv (oas = 100bp) = %.6f" % swaption3.NPV())

Bond's bermudan call right npv (oas = 100bp) = 0.044980


The next instrument we look at is a CMS 10Y vs Euribor 6M swaption. The maturity is again 10 years and the option is exercisable on a yearly basis

In [78]:
underlying4 = ql.FloatFloatSwap(ql.VanillaSwap.Payer, [1.0]*(len(fixedSchedule)-1), [1.0]*(len(floatingSchedule)-1), fixedSchedule,
                               swapBase, ql.Thirty360(), floatingSchedule,
                               euribor6m, ql.Actual360(), False, False,
                               [1.0]*(len(fixedSchedule)-1),[0.0]*(len(fixedSchedule)-1),[] ,[] ,[1.0]*(len(floatingSchedule)-1),
                                [0.001]*(len(floatingSchedule)-1),[],[])

In [79]:
swaption4 = ql.FloatFloatSwaption(underlying4, exercise)

In [80]:
floatSwaptionEngine = ql.Gaussian1dFloatFloatSwaptionEngine(gsr, 64, 7.0, True, False, ql.QuoteHandle(), ytsOis, True, ql.Gaussian1dFloatFloatSwaptionEngine.Naive)

In [81]:
swaption4.setPricingEngine(floatSwaptionEngine)

Since the underlying is quite exotic already, we start with pricing this using the LinearTsrPricer for CMS coupon estimation

In [82]:
reversionQuote=ql.QuoteHandle(ql.SimpleQuote(reversion))

In [83]:
leg0 = underlying4.leg(0)
leg1 = underlying4.leg(1)

In [84]:
cmsPricer = ql.LinearTsrPricer(swaptionVolHandle, reversionQuote)

In [85]:
ql.setCouponPricer(leg0,cmsPricer)

In [86]:
iborPricer = ql.BlackIborCouponPricer()

In [87]:
ql.setCouponPricer(leg1,iborPricer)

In [88]:
swapPricer = ql.DiscountingSwapEngine(ytsOis)

In [89]:
underlying4.setPricingEngine(swapPricer)

In [90]:
npv5 = underlying4.NPV()

In [91]:
print("Underlying CMS Swap NPV = %.6f" % npv5)

Underlying CMS Swap NPV = 0.004447


In [92]:
print("       CMS     Leg  NPV = %.6f" % underlying4.legNPV(0))

       CMS     Leg  NPV = -0.231736


In [93]:
print("       Euribor Leg  NPV = %.6f" % underlying4.legNPV(1))

       Euribor Leg  NPV = 0.236183


We generate a naive calibration basket and calibrate the GSR model to it:

In [94]:
basket = swaption4.calibrationBasket(swapBase, swaptionVol, "Naive");

In [95]:
for helper in basket:
    helper.setPricingEngine(swaptionEngine)

In [96]:
gsr.calibrateVolatilitiesIterative(basket, method, ec)

In [97]:
printBasket(basket)


Expiry              Maturity            Nominal             Rate          Pay/Rec     Market ivol   
April 30th, 2015    May 6th, 2024       1.00                0.025307      0           0.200000      
May 3rd, 2016       May 6th, 2024       1.00                0.025300      0           0.200000      
May 3rd, 2017       May 6th, 2024       1.00                0.025303      0           0.200000      
May 3rd, 2018       May 6th, 2024       1.00                0.025306      0           0.200000      
May 2nd, 2019       May 6th, 2024       1.00                0.025311      0           0.200000      
April 30th, 2020    May 6th, 2024       1.00                0.025300      0           0.200000      
May 3rd, 2021       May 6th, 2024       1.00                0.025306      0           0.200000      
May 3rd, 2022       May 6th, 2024       1.00                0.025318      0           0.200000      
May 3rd, 2023       May 6th, 2024       1.00                0.025353      0           0.20

In [98]:
printModelCalibration(basket, gsr.volatility())


Expiry              Model sigma   Model price         Market price        Model ivol    Market ivol   
April 30th, 2015    0.005178      0.016111            0.016111            0.200000      0.200000      
May 3rd, 2016       0.005156      0.020062            0.020062            0.200000      0.200000      
May 3rd, 2017       0.005149      0.021229            0.021229            0.200000      0.200000      
May 3rd, 2018       0.005129      0.020738            0.020738            0.200000      0.200000      
May 2nd, 2019       0.005132      0.019096            0.019096            0.200000      0.200000      
April 30th, 2020    0.005074      0.016537            0.016537            0.200000      0.200000      
May 3rd, 2021       0.005091      0.013253            0.013253            0.200000      0.200000      
May 3rd, 2022       0.005097      0.009342            0.009342            0.200000      0.200000      
May 3rd, 2023       0.005001      0.004910            0.004910          

In [99]:
npv6 = swaption4.NPV()

The npv of the bermudan swaption is

In [100]:
print("Float swaption NPV (GSR) = %.6f" % npv6)

Float swaption NPV (GSR) = 0.004291


In this case it is also interesting to look at the underlying swap npv in the GSR model.

In [101]:
print("Float swap NPV (GSR) = %.6f" % swaption4.underlyingValue())

Float swap NPV (GSR) = 0.005250


In [102]:
probResult = swaption4.probabilities()

In [103]:
print("Call probabilities (Naive):")
for prob in probResult:
    print ("%.4f" % prob)

Call probabilities (Naive):
0.9300
0.0141
0.0058
0.0032
0.0025
0.0012
0.0010
0.0012
0.0012
0.0398


In [104]:
floatSwaptionEngine2 = ql.Gaussian1dFloatFloatSwaptionEngine(gsr, 64, 7.0, True, False, ql.QuoteHandle(), ytsOis, True, ql.Gaussian1dFloatFloatSwaptionEngine.Digital)

In [105]:
swaption4.setPricingEngine(floatSwaptionEngine2)
probResult2 = swaption4.probabilities()

In [106]:
print("Call probabilities (Digital):")
for prob in probResult2:
    print ("%.4f" % prob)

Call probabilities (Digital):
0.9560
0.0123
0.0053
0.0031
0.0025
0.0012
0.0011
0.0014
0.0013
0.0192


Not surprisingly, the underlying is priced differently compared to the LinearTsrPricer, since a different smile is implied by the GSR model.

This is exactly where the Markov functional model comes into play, because it can calibrate to any given underlying smile (as long as it is arbitrage free). We try this now. Of course the usual use case is not to calibrate to a flat smile as in our simple example, still it should be possible, of course...

In [107]:
markovStepDates = exerciseDates

In [108]:
cmsFixingDates = markovStepDates

In [109]:
markovSigmas = [0.01 for i in range(0, len(markovStepDates)+1)]

In [110]:
tenors = [ql.Period(10,ql.Years) for i in cmsFixingDates]

In [111]:
settings = ql.MarkovFunctionalSettings(16, 7.0, 32, 1e-5, 1e-7, 0.0, 2.0, ql.MarkovFunctionalSettings.KahaleSmile | ql.MarkovFunctionalSettings.SmileExponentialExtrapolation)

In [112]:
settings.validate()

In [113]:
markov = ql.MarkovFunctional(yts6m, reversion, markovStepDates, markovSigmas, swaptionVolHandle,
                            cmsFixingDates, tenors, swapBase, settings)

In [114]:
swaptionEngineMarkov = ql.Gaussian1dSwaptionEngine(markov, 8, 5.0, True, False, ytsOis)

In [115]:
floatEngineMarkov = ql.Gaussian1dFloatFloatSwaptionEngine(markov, 16, 7.0, True, False, ql.QuoteHandle(), ytsOis, True, ql.Gaussian1dFloatFloatSwaptionEngine.Naive)

In [116]:
swaption4.setPricingEngine(floatEngineMarkov)

In [117]:
npv7 = swaption4.NPV()

The option npv is the markov model is:

In [118]:
print("Float swaption NPV (Markov) = %.6f" % npv7)

Float swaption NPV (Markov) = 0.003549


This is not too far from the GSR price.

More interesting is the question how well the Markov model did its job to match our input smile. For this we look at the underlying npv under the Markov model

In [119]:
print("Float swap NPV (Markov) = %.6f" % swaption4.underlyingValue())

Float swap NPV (Markov) = 0.004301


In [120]:
probResult3 = swaption4.probabilities()
print("Call probabilities (Digital):")
for prob in probResult3:
    print ("%.4f" % prob)

Call probabilities (Digital):
0.8617
0.0369
0.0235
0.0144
0.0097
0.0070
0.0054
0.0000
0.0338
0.0068


This is closer to our terminal swap rate model price. A perfect match is not expected anyway, because the dynamics of the underlying rate in the linear model is different from the Markov model, of course.

The Markov model can not only calibrate to the underlying smile, but has at the same time a sigma function (similar to the GSR model) which can be used to calibrate to a second instrument set. We do this here to calibrate to our coterminal ATM swaptions from above.

This is a computationally demanding task, so depending on your machine, this may take a while now...

In [121]:
l=ql.FixedRateLeg(fixedSchedule, ql.Actual360(),[1.0],[0.01])

In [122]:
for helper in basket:
    helper.setPricingEngine(swaptionEngineMarkov)

In [123]:
markov.calibrate(basket, method, ec)

In [124]:
printModelCalibration(basket, markov.volatility())


Expiry              Model sigma   Model price         Market price        Model ivol    Market ivol   
April 30th, 2015    0.010000      0.016111            0.016111            0.199997      0.200000      
May 3rd, 2016       0.012276      0.020062            0.020062            0.200002      0.200000      
May 3rd, 2017       0.010534      0.021229            0.021229            0.200001      0.200000      
May 3rd, 2018       0.010414      0.020738            0.020738            0.200001      0.200000      
May 2nd, 2019       0.010361      0.019096            0.019096            0.199999      0.200000      
April 30th, 2020    0.010339      0.016537            0.016537            0.200001      0.200000      
May 3rd, 2021       0.010365      0.013253            0.013253            0.199999      0.200000      
May 3rd, 2022       0.010382      0.009342            0.009342            0.200001      0.200000      
May 3rd, 2023       0.010392      0.004910            0.004910          

Now let's have a look again at the underlying pricing. It shouldn't have changed much, because the underlying smile is still matched.

In [125]:
npv8 = swaption4.underlyingValue()

In [126]:
print("Float swap NPV (Markov) = %.6f" % npv8)

Float swap NPV (Markov) = 0.004331


This is close to the previous value as expected.

As a final remark we note that the calibration to coterminal swaptions is not particularly reasonable here, because the european call rights are not well represented by these swaptions. Secondly, our CMS swaption is sensitive to the correlation between the 10y swap rate and the Euribor 6M rate. Since the Markov model is one factor by construction.

In [127]:
params=gsr.params()

In [128]:
for p in params:
    print(p)

0.01
0.005178115995991178
0.005156428197321668
0.005149315186589236
0.005129242647787604
0.005132423720732759
0.0050744519243368594
0.005091184196963047
0.005096812297853889
0.005001398858149273


In [129]:
gsr.setParams(params)

In [130]:
gsr.constraint()==ql.NoConstraint()

False

In [131]:
gsr.endCriteria()

3

In [132]:
for v in gsr.problemValues():
    print(v)

1.3072676417292392e-14


In [133]:
gsr.functionEvaluation()

10

In [134]:
params2=markov.params()

In [135]:
for p in params2:
    print(p)

0.01
0.012275967609634385
0.010534392133047691
0.010413614142191998
0.010360865663812238
0.0103394663701287
0.010365226214887812
0.010381804288863332
0.010392045889194344
0.009959089531755682


In [136]:
markov.endCriteria()

3

In [137]:
for v in markov.problemValues():
    print(v)

1.747469034039124e-05
9.208198024982184e-06
2.735093083210386e-06
5.151509390029682e-06
3.9362171751408114e-06
5.538244386530715e-06
4.7290011162726084e-06
6.14406286222487e-06
9.667031541107928e-07


In [138]:
markov.functionEvaluation()

193